# Simulation of Condor

This is a simple simulation about users accessing condor services and how this could produce different queues.

## 1. First Step

Get time metrics to define events.

In [1]:
# Constants of the simulation

SERVERS = 3
SERVER_1 = {
    "enroll": {"type": "uniform", "time": [2, 5]},
    "check": {"type": "normal", "time": {"mu": 5, "sigma": 1}},
}

SERVER_2 = {
    "enroll": {"type": "uniform", "time": [3, 7]},
    "check": {"type": "normal", "time": {"mu": 3, "sigma": 1}},
}

SERVER_3 = {
    "enroll": {"type": "normal", "time": {"mu": 6, "sigma": 1}},
    "check": {"type": "uniform", "time": [3, 6]},
}

STUDENTS_ARRIVAL = {
    "event": {"type": "normal", "time": {"mu": 3, "sigma": 1}},
    "action": {"enroll": 0.6, "check": 0.4},
}

## 2. Second Step: Generate Random Variables

Here some functions sdould be used to generate random numbers based on distributions.

In [2]:
import random


def generate_uniform_random(min_time: int, max_time: int) -> int:
    """This function generates a random number based
    on both lower and upper bounds.

    Args:
        min_time (int): Lower bound
        max_time (int): Upper bound

    Returns:
        int: Random number
    """
    return round(min_time + (max_time - min_time) * random.random())


def generate_normal_random(mu: int, sigma: int) -> int:
    """This function generates a random number based
    on normal distribution.

    Args:
        mu (int): Mean
        sigma (int): Standard deviation

    Returns:
        int: Random number
    """
    return round(random.gauss(mu, sigma))

## Third Step: Define Queue and Events

In [3]:
# Entity class
class Student:
    def __init__(
        self, id: int, arrival_time: int, type_action: str
    ):
        self.id = id
        self.arrival_time = arrival_time
        self.type_action = type_action

class Server:
    def __init__(self, name: str, distribution: dict):
        self.name = name
        self.distribution = distribution
        self.busy = False

In [4]:
def next_student_arrival(current_time: int) -> int:
    distribution = STUDENTS_ARRIVAL["event"]["type"]
    if distribution == "uniform":
        min = STUDENTS_ARRIVAL["event"]["time"][0]
        max = STUDENTS_ARRIVAL["event"]["time"][1]
        uniform_random = generate_uniform_random(min, max)
        return current_time + uniform_random
    elif distribution == "normal":
        mu = STUDENTS_ARRIVAL["event"]["time"]["mu"]
        sigma = STUDENTS_ARRIVAL["event"]["time"]["sigma"]
        normal_random = generate_normal_random(mu, sigma)
        return current_time + normal_random

In [ ]:
students_queue = []
servers = [Server("s1", SERVER_1), Server("s2", SERVER_2), Server("s3", SERVER_3)]

time = 0
limit_time = 200
id_student = 1

EVENTS = {
    "student_arrival": next_student_arrival(time),
    "server_1_available": -1,
    "server_2_available": -1,
    "server_3_available": -1
}


while time <= limit_time:
    if EVENTS["student_arrival"] == time:
        student = Student(
            id=id_student,
            arrival_time=time,
            type_action="enroll" if random.random() < STUDENTS_ARRIVAL["action"]["enroll"] else "check"
        )
        students_queue.append(student)
        EVENTS["student_arrival"] = next_student_arrival(time)    
        id_student += 1

    if servers[0].busy and EVENTS["server_1_available"] == time:
        servers[0].busy = False
    if servers[1].busy and EVENTS["server_2_available"] == time:
        servers[1].busy = False
    if servers[2].busy and EVENTS["server_3_available"] == time:
        servers[2].busy = False

    if len(students_queue) > 0:
        

    time += 1